In [1]:
import os
import csv
import numpy as np
# import frgtools.misc as frgm
import matplotlib.pyplot as plt

In [ ]:
fids = frgm.listdir('Data')

In [4]:
filepath = r"C:\Users\Skaggs\Desktop\Perovskites\2019_08_09 CV SnOx\10nm\180_11.4_FTO_02_CV_C02.txt"

getCV(filepath, area = 0.1)


{'v': array([0.19229576, 0.19345422, 0.19439143, ..., 0.19039437, 0.19128586,
        0.19239724]),
 'j': array([0.00034713, 0.00034614, 0.00034744, ..., 0.00037424, 0.00037562,
        0.00040446]),
 'numscans': None,
 'area': 1,
 'scans': {'mode': [array([2., 2., 2., ..., 2., 2., 2.]),
   array([2., 2., 2., ..., 2., 2., 2.]),
   array([2., 2., 2., ..., 2., 2., 2.]),
   array([2., 2., 2., ..., 2., 2., 2.])],
  'ox/red': [array([1., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 1., 1., 1.])],
  'error': [array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.])],
  'control changes': [array([0., 1., 1., ..., 1., 1., 1.]),
   array([0., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 0., 0., 0.])],
  'counter inc.': [array([0., 0., 0., ..., 1., 1., 1.]

In [2]:
# getCV
"""
Loads CV data from a text file. Pulls the last curve out of the
repeats (red/ox cycle).
Returns structure. 

Example: mydata = getCV(use UI to select file)

		 mydata = getCV ('C:\Myname\CV\File.ras')   (directly offer fpath)

Output: Structure with J and V from the final run, along with all other associated data in text file.
    mydata['v'] --> voltage
    mydata['j'] --> current density
        ['header'] --> all info from the header
        ['area']
        ['date']
        ['time']
        ['num_cycles']
        ['cathodic_peak'] --> onset potential method 1
        ['anodic_peak'] --> anodic peak current (at the right most end (highest positive potential))
        ['inflection_onset'] --> onset method 2
        ['intersect_onset'] --> onset method 3
        ['scans'] --> 
            [j][#]
            [v][#]
            
            
Onset potential methods: 
	1: Direct minimum
	2: Inflection point
	3: Intersection at baseline

"""
import os
import numpy as np
import csv
# from .plotting import directionalplot

def getCV(fpath, area =1):
    cvdata = load_cv(fpath)
    cvdata = getOnsetPotentials(cvdata)
    return cvdata

def load_cv(fpath, area = 1):
	# headerkey = {
	#     'Run on channel': 'channel',
	#     'Electrode connection': 'electrode_connection',
	#     'Ewe,I filtering': 'ewe_ifilter',
	#     'Channel': 'channel',
	#     'Acquisition started on': 'datetime',
	#     'Reference electrode': 'ref_electrode',
	# }


	DATA_START = 'mode\tox/red\terror'

	data = {
		'v': None,
		'j': None,
		'numscans': None,
		'area': area,
		'scans': {},
		'header': {}
	}

	key = None
	previouskey = None
	with open(fpath, 'r', errors = 'ignore', encoding = 'utf-8') as f:
		headerlines = 54 #initial guess, will get picked up in code at 
		line = f.readline()#.decode('utf-8')
		while not line.startswith(DATA_START):
			if line.startswith('Acquisition started on'):
				data['date'], data['time'] = line.split(' : ')[1].split(' ')
			else:
				if ' : ' in line:
					key, value = line.split(' : ')
					data['header'][key.strip()] = value.strip()
				elif '  ' in line:
					parts = line.strip().split('  ')
					key = parts[0]
					value = parts[-1]
					if key == 'vs.':
						data['header'][previouskey] += ' vs. {}'.format(value.strip())
					else:
						data['header'][key.strip()] = value.strip()
				previouskey = key
			line = f.readline()#.decode('utf-8')
		
		colnames = []
		for c in line.strip().split('\t'):
			if c == 'ox/red':
				colnames.append(c)
			elif c.startswith('<I>'):
				colnames.append('i')
			elif c.startswith('Ewe'):
				colnames.append('v')
			elif c.startswith('cycle'):
				colnames.append('cycle')
			else:
				colnames.append(c.split('/')[0])
		f_reader = csv.DictReader(f, fieldnames = colnames, delimiter = '\t')  
		data['scans'] = {k:[[]] for k in colnames}
		currentcycle = 1
		for row in f_reader:
			if int(row['cycle']) > currentcycle:
				currentcycle = int(row['cycle'])
				for k in data['scans'].keys():
					data['scans'][k].append([])
			for k, val in row.items():
				data['scans'][k][currentcycle-1].append(float(val))

		data['scans'] = {k:[np.array(cycledata) for cycledata in v] for k,v in data['scans'].items()}
		data['scans']['j'] = [i/area for i in data['scans']['i']]
		data['num_cycles'] = currentcycle
		data['v'] = data['scans']['v'][-1]
		data['j'] = data['scans']['j'][-1]

		return data

def getOnsetPotentials(cvdata,fitwidth = 40):
    # Finding cathodic peak voltage
    X = cvdata['v']
    Y = cvdata['j']
    idx0 = np.argmin(Y)
    cvdata['cathodic_peak'] = X[idx0] 
    
    # Finding anodic peak current at the end of the voltammogram
    idx1 = np.argmax(X)
    cvdata['anodic_peak'] = Y[idx1] 

    # Inflection point onset
    idx2 = np.argmin(X)
    XX = X[idx1+500 : idx2 - 20]
    YY = Y[idx1+500 : idx2 - 20]
    
    dydx = np.gradient(YY)/np.gradient(XX) # Finds first derivative, delta y/ delta x
    dydx = smooth(dydx)
    
    idx3 = np.argmax(dydx) # Inflection point on the cathodic curve
    idx4 = np.argmin(abs(YY)) # Getting closest to the baseline center as possible
    cvdata['inflection_onset'] = XX[idx3]
    
    X_ = XX[idx3 - fitwidth//2: idx3 + fitwidth//2]
    Y_ = YY[idx3 - fitwidth//2: idx3 + fitwidth//2]
    linfit_3 = np.polyfit(X_,Y_,1)
    
    X_ = XX[idx4 - fitwidth//2: idx4 + fitwidth//2]
    Y_ = YY[idx4 - fitwidth//2: idx4 + fitwidth//2]
    linfit_4 = np.polyfit(X_,Y_,1)
    
    intersect_x = (linfit_4[1] - linfit_3[1]) / (linfit_3[0] - linfit_4[1])
    intersect_y = linfit_3[0] * intersect_x + linfit_3[1]
    cvdata['intersect_onset'] = [intersect_x, intersect_y]
    return cvdata

def smooth(a,WSZ=5):
    # a: NumPy 1-D array containing the data to be smoothed
    # WSZ: smoothing window size needs, which must be odd number,
    # as in the original MATLAB implementation
    out0 = np.convolve(a,np.ones(WSZ,dtype=int),'valid')/WSZ    
    r = np.arange(1,WSZ-1,2)
    start = np.cumsum(a[:WSZ-1])[::2]/r
    stop = (np.cumsum(a[:-WSZ:-1])[::2]/r)[::-1]
    return np.concatenate((  start , out0, stop  ))

In [5]:
# getCV
"""
Loads CV data from a text file. Pulls the last curve out of the
repeats (red/ox cycle).
Returns structure. 
area is in cm^2

Example: mydata = getCV     (use UI to select file)

         mydata = getCV ('C:\Myname\CV\File.ras')   (directly offer filepath)

Onset potential methods: 
    1: Direct voltage at minimum cathodic peak
    2: Inflection point
    3: Intersection at baseline
    4: Fixed
"""


def getcvdata(filepath, area = 1, nheaderlines = 53, getall = True):
    area = float(area)
    allCV = {'voltage' : np.array([],dtype = float),
             'current' : np.array([],dtype = float),
             'cycle'   : np.array([],dtype = float)}

    with open(filepath,'r') as f:
        [next(f) for item in range(nheaderlines)] # Skips until the 54th line, where header line is. 
        f_reader = csv.DictReader(f, delimiter = '\t')    
        for row in f_reader:
            allCV['voltage'] = np.append(allCV['voltage'],float(row['Ewe/V']))
            allCV['current'] = np.append(allCV['current'],float(row['<I>/mA']))
            allCV['cycle'] = np.append(allCV['cycle'],float(row['cycle number']))
    allCV['currentdensity'] = np.divide(allCV['current'], area)
    
    num_of_scans = np.unique(allCV['cycle'])
    fs = num_of_scans[-1]
    final_scan = {'date' : [],
                'voltage' : [],
                'current' : [],
                'currentdensity' : []}

    j = 0  # This can probably be optimized. Final scan is the only important one.
    for n in allCV['cycle']:
        if n == fs:
            final_scan['voltage'] = np.append(final_scan['voltage'], allCV['voltage'][j])
            final_scan['current'] = np.append(final_scan['current'], allCV['current'][j])
            final_scan['currentdensity'] = np.append(final_scan['currentdensity'], allCV['currentdensity'][j])
        j += 1
        
    final_scan['allCV'] = allCV # allCV is a struct with all cyclenumbers. 
    return final_scan

def smooth(a,WSZ=5):
    # a: NumPy 1-D array containing the data to be smoothed
    # WSZ: smoothing window size needs, which must be odd number,
    # as in the original MATLAB implementation
    out0 = np.convolve(a,np.ones(WSZ,dtype=int),'valid')/WSZ    
    r = np.arange(1,WSZ-1,2)
    start = np.cumsum(a[:WSZ-1])[::2]/r
    stop = (np.cumsum(a[:-WSZ:-1])[::2]/r)[::-1]
    return np.concatenate((  start , out0, stop  ))

def getOnsetPotentials(cvdata,fitwidth = 40):
    #cvdata should be dict, output from getCV
    
    # Finding cathodic peak voltage
    X = cvdata['v']
    Y = cvdata['j']
    idx0 = np.argmin(Y)
    cvdata['cathodic_peak'] = X[idx0] 
    
    # Finding anodic peak current at the end of the voltammogram
    idx1 = np.argmax(X)
    cvdata['anodic_peak'] = Y[idx1] 

    # Inflection point onset
    idx2 = np.argmin(X)
    XX = X[idx1+500 : idx2 - 20]
    YY = Y[idx1+500 : idx2 - 20]
    
    dydx = np.gradient(YY)/np.gradient(XX) # Finds first derivative, delta y/ delta x
    dydx = smooth(dydx)
    
    idx3 = np.argmax(dydx) # Inflection point on the cathodic curve
    idx4 = np.argmin(abs(YY)) # Getting closest to the baseline center as possible
    cvdata['inflection_onset'] = XX[idx3]
    
    X_ = XX[idx3 - fitwidth//2: idx3 + fitwidth//2]
    Y_ = YY[idx3 - fitwidth//2: idx3 + fitwidth//2]
    linfit_3 = np.polyfit(X_,Y_,1)
    
    X_ = XX[idx4 - fitwidth//2: idx4 + fitwidth//2]
    Y_ = YY[idx4 - fitwidth//2: idx4 + fitwidth//2]
    linfit_4 = np.polyfit(X_,Y_,1)
    
    intersect_x = (linfit_4[1] - linfit_3[1]) / (linfit_3[0] - linfit_4[1])
    intersect_y = linfit_3[0] * intersect_x + linfit_3[1]
    cvdata['intersect_onset'] = [intersect_x, intersect_y]
    return cvdata

def getCV(fpath, area =1):
    cvdata = getcvdata(fpath)
    cvdata = getOnsetPotentials(cvdata)
    return cvdata

In [32]:
filepath = r"C:\Users\Skaggs\Desktop\Perovskites\2019_08_09 CV SnOx\10nm\180_11.4_FTO_02_CV_C02.txt"
getCV(filepath, area = 0.1)

{'date': [],
 'voltage': array([0.19229576, 0.19345422, 0.19439143, ..., 0.19039437, 0.19128586,
        0.19239724]),
 'current': array([0.00034713, 0.00034614, 0.00034744, ..., 0.00037424, 0.00037562,
        0.00040446]),
 'currentdensity': array([0.00034713, 0.00034614, 0.00034744, ..., 0.00037424, 0.00037562,
        0.00040446]),
 'allCV': {'voltage': array([0.19221958, 0.19306683, 0.19405744, ..., 0.19039437, 0.19128586,
         0.19239724]),
  'current': array([-6.26124907e-04, -2.78200574e-04, -8.87431438e-05, ...,
          3.74235361e-04,  3.75616966e-04,  4.04460821e-04]),
  'cycle': array([1., 1., 1., ..., 4., 4., 4.]),
  'currentdensity': array([-6.26124907e-04, -2.78200574e-04, -8.87431438e-05, ...,
          3.74235361e-04,  3.75616966e-04,  4.04460821e-04])},
 'cathodic_peak': -0.444778711,
 'anodic_peak': 0.010884792647427984,
 'inflection_onset': -0.309950382,
 'intersect_onset': [-0.18963104576730827, -0.00028764459757954497]}

In [35]:
ans = getCV(filepath, area = 0.1)
print(len(ans['voltage']))
print(len(ans['allCV']['voltage']))

4199
16797


In [ ]:
headerkey = {
    'Run on channel': 'channel',
    'Electrode connection': 'electrode_connection',
    'Ewe,I filtering': 'ewe_ifilter',
    'Channel': 'channel',
    'Acquisition started on': 'datetime',
    'Reference electrode': 'ref_electrode',
}

DATA_START = 'mode\tox/red\terror'
NUM_HEADERLINES = 'Nb header lines':

{'v': array([0.19229576, 0.19345422, 0.19439143, ..., 0.19039437, 0.19128586,
        0.19239724]),
 'j': array([0.00347129, 0.0034614 , 0.00347438, ..., 0.00374235, 0.00375617,
        0.00404461]),
 'numscans': None,
 'area': 0.1,
 'scans': {'mode': [array([2., 2., 2., ..., 2., 2., 2.]),
   array([2., 2., 2., ..., 2., 2., 2.]),
   array([2., 2., 2., ..., 2., 2., 2.]),
   array([2., 2., 2., ..., 2., 2., 2.])],
  'ox/red': [array([1., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 1., 1., 1.])],
  'error': [array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.])],
  'control changes': [array([0., 1., 1., ..., 1., 1., 1.]),
   array([0., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 1., 1., 1.]),
   array([1., 1., 1., ..., 0., 0., 0.])],
  'counter inc.': [array([0., 0., 0., ..., 1., 1., 1

In [ ]:
area = 1

data = {
    'cv_raw': {}
}
with open(fids[0], 'r', errors = 'ignore', encoding = 'utf-8') as f:
    headerlines = 54 #initial guess, will get picked up in code at 
    line = f.readline()#.decode('utf-8')
    while not line.startswith(DATA_START):
        for k, v in headerkey.items():
            if line.startswith(k):
                decodedline = line
                data[v] = decodedline.split(' : ')[-1].replace('\n', '')
        line = f.readline()#.decode('utf-8')
    
    colnames = []
    for c in line.strip().split('\t'):
        if c == 'ox/red':
            colnames.append(c)
        elif c.startswith('<I>'):
            colnames.append('i')
        elif c.startswith('control'):
            colnames.append('v')
        elif c.startswith('cycle'):
            colnames.append('cycle')
        else:
            colnames.append(c.split('/')[0])
    f_reader = csv.DictReader(f, fieldnames = colnames, delimiter = '\t')    
    for row in f_reader:
        if len(data['cv_raw']) == 0:
            data['cv_raw'] = {k:[] for k in row.keys()}
        for k, v in row.items():
            data['cv_raw'][k].append(float(v))
    
    data['cv_raw'] = {k:np.array(v) for k,v in data['cv_raw'].items()}
    
    
    mask = data['cv_raw']['cycle'] == data['cv_raw']['cycle'].max()
    
    data['v'] = data['cv_raw']['v'][mask]
    data['j'] = data['cv_raw']['i'][mask]/area
    